In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

In [4]:
import os

In [5]:
trial_model=load_model('VGGFace.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








C:\Users\Om\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [7]:
def feature_extractor(inpDir,personID,wordID,timestep,ex_type=True): #access directory and extract face vector
    
    
    if ex_type == True:
        Y_train_labels=np.ones((10,512))
    else:
        Y_train_labels=np.zeros((10,512))  #remove after training stage is completed
        
        
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir=inpDir #input dataset
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[personID]
    personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
    lpersonFolder = os.listdir(personFolder)
    wordID= lpersonFolder[wordID]
    wordFolder = '%s\\%s' % (personFolder,wordID)
    lwordFolder = os.listdir(wordFolder)
    
   
    i=0
    for utterance in lwordFolder:
        
        utterFolder= '%s\\%s' % (wordFolder,utterance)
        lutterFolder = os.listdir(utterFolder)
        colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
        lcolourFolder = os.listdir(colourFolder)
        frame = lcolourFolder[timestep]
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
            
    print("Extraction completed")
    
    WB_CNN= Sequential()
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation= 'relu', input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation='relu',input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(512, kernel_size=(1,1), activation='relu'))
    WB_CNN.add(Flatten())   

    WB_CNN.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    WB_CNN.fit(feature_sequence,Y_train_labels,epochs=20,batch_size=2)
    CNN_out = WB_CNN.predict(feature_sequence)
    
    print("Prediction done")
    print(CNN_out.shape)
    return CNN_out   

In [8]:
CNN_out_test=feature_extractor('C:\\Users\\Om\\Desktop\\Dataset',0,0,0,True)

Extraction completed


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
10/10 [==============================] - 3s 287ms/step - loss: 6.6105 - acc: 0.0953
Epoch 2/20
10/10 [==============================] - 1s 54ms/step - loss: 4.2594 - acc: 0.0246
Epoch 3/20
10/10 [==============================] - 1s 51ms/step - loss: 3.4404 - acc: 0.0102
Epoch 4/20
10/10 [==============================] - 1s 52ms/step - loss: 2.8531 - acc: 0.0037
Epoch 5/20
10/10 [==============================] - 1s 60ms/step - loss: 2.5328 - acc: 0.0045
Epoch 6/20
10/10 [==============================] - 1s 58ms/step - loss: 2.4328 - acc: 0.0018
Epoch 7/20
10/10 [==============================] - 1s 52ms/step - loss: 2.1483 - acc: 0.0021
Epoch 8/20
10/10 [==============================] - 1s 55ms/step - loss: 2.0211 - acc: 0.0016
Epoch 9/20
10/10 [==============================] - 1s 52ms/step - loss: 1.9187 - acc: 0.0014
Epoch 10/20
10/10 [===============

In [10]:
def per_to_feat(inpDir,personID,wordID,ex_type=True):
    timesteps=11
    count=0
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,personID,wordID,i,True)
        count = count + 1
        if count==1 :
            CNN_out_total= CNN_out #maintaining same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
    print('Concatenation completed')
    return CNN_out_total      

In [11]:
CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',0,0,True)

Extraction completed
Epoch 1/20
10/10 [==============================] - 3s 313ms/step - loss: 6.6767 - acc: 0.0904
Epoch 2/20
10/10 [==============================] - 1s 53ms/step - loss: 4.2433 - acc: 0.0242
Epoch 3/20
10/10 [==============================] - 1s 53ms/step - loss: 3.2096 - acc: 0.0104
Epoch 4/20
10/10 [==============================] - 1s 54ms/step - loss: 2.6351 - acc: 0.0078
Epoch 5/20
10/10 [==============================] - 1s 55ms/step - loss: 2.3367 - acc: 0.0047
Epoch 6/20
10/10 [==============================] - 1s 55ms/step - loss: 2.1840 - acc: 0.0027
Epoch 7/20
10/10 [==============================] - 1s 58ms/step - loss: 1.9680 - acc: 0.0027
Epoch 8/20
10/10 [==============================] - 1s 55ms/step - loss: 1.8070 - acc: 0.0012
Epoch 9/20
10/10 [==============================] - 1s 54ms/step - loss: 1.6262 - acc: 9.7656e-04
Epoch 10/20
10/10 [==============================] - 1s 53ms/step - loss: 1.5051 - acc: 0.0016
Epoch 11/20
10/10 [==============

10/10 [==============================] - 1s 74ms/step - loss: 2.4429 - acc: 0.0043
Epoch 5/20
10/10 [==============================] - 1s 64ms/step - loss: 2.1283 - acc: 0.0049
Epoch 6/20
10/10 [==============================] - 1s 57ms/step - loss: 1.8667 - acc: 0.0031
Epoch 7/20
10/10 [==============================] - 1s 61ms/step - loss: 1.8012 - acc: 0.0023
Epoch 8/20
10/10 [==============================] - 1s 59ms/step - loss: 1.6792 - acc: 0.0021
Epoch 9/20
10/10 [==============================] - 1s 68ms/step - loss: 1.5856 - acc: 0.0020
Epoch 10/20
10/10 [==============================] - 1s 58ms/step - loss: 1.4661 - acc: 0.0014
Epoch 11/20
10/10 [==============================] - 1s 54ms/step - loss: 1.3804 - acc: 0.0023
Epoch 12/20
10/10 [==============================] - 1s 53ms/step - loss: 1.2631 - acc: 7.8125e-04
Epoch 13/20
10/10 [==============================] - 1s 55ms/step - loss: 1.1878 - acc: 9.7656e-04
Epoch 14/20
10/10 [==============================] - 1s 54m

In [24]:
print(CNN_out_total)

[[102.750374 103.97822   51.516937 ... 121.622     83.35776   54.51599 ]
 [ 93.41889   93.78788   46.74794  ... 109.03468   75.138466  50.038185]
 [ 92.235275  92.97389   46.68425  ... 108.108765  74.5548    49.783627]
 ...
 [ 53.730087  20.760727  34.968376 ...  34.01365   21.93544   33.950058]
 [ 53.730087  20.760727  34.968376 ...  34.01365   21.93544   33.950058]
 [ 53.730087  20.760727  34.968376 ...  34.01365   21.93544   33.950058]]


In [31]:
def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    count_i=0
    for i in range(10):
        count_j=0
        for j in range(0,110,10):
            j=j+i
            sample_mat_j=sample_mat[j]
            sample_mat_j=sample_mat_j.reshape((1,512))
            if count_j==0:
                feature_vector=sample_mat_j
            else:
                feature_vector=np.concatenate((feature_vector,sample_mat_j))
            count_j = count_j + 1
        
        if count_i==0:
            feature_mat=feature_vector
        else:
            feature_mat= np.concatenate((feature_mat,feature_vector))
        count_i= count_i + 1
    LSTM_input= feature_mat
    return LSTM_input

In [34]:
LSTM_input= reshape_LSTM(CNN_out_total)
print(LSTM_input.shape)
LSTM_input= LSTM_input.reshape((10,11,512))


(110, 512)


In [35]:
print(LSTM_input.shape)

(10, 11, 512)


In [36]:
from numpy import save

In [37]:
save('The_woman.npy', LSTM_input)
